<a href="https://colab.research.google.com/github/criswd/projeto_drone/blob/main/Detect_Objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de Objetos com YOLOv5

Este notebook documenta o processo de treinamento e detecção de objetos utilizando o modelo YOLOv5. O objetivo é treinar um modelo personalizado para identificar telhados e pessoas em imagens aéreas capturadas por drones. O modelo foi treinado com um conjunto específico de dados rotulados e configurado para maximizar a precisão da detecção em imagens novas e não vistas anteriormente.

Neste projeto, utilizamos o modelo YOLOv5, uma das arquiteturas de detecção de objetos mais avançadas, para identificar objetos específicos em imagens capturadas por drones. O objetivo é desenvolver um sistema capaz de automatizar a identificação de elementos como telhados e pessoas, proporcionando uma ferramenta útil para tarefas de monitoramento e inspeção aérea.

## Objetivos
- Treinar o modelo YOLOv5 com imagens personalizadas.
- Avaliar a performance do modelo treinado.
- Utilizar o modelo para detectar objetos em novas imagens.


### Etapa 1: Configuração do Ambiente
Inicialmente, configuramos o ambiente de desenvolvimento no Google Colab, instalando as dependências necessárias e preparando a estrutura de diretórios.


In [ ]:
#montando acesso ao googleDrive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#clonando projeto
! git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 23.87 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.


In [ ]:
#instalando requerimentos do reconhecedor de objetos

! pip install -r yolov5/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.1/877.1 kB 22.9 MB/s eta 0:00:00


### Etapa 2: Preparação dos dados e Configuração do Modelo
Os dados foram coletados, rotulados e organizados em pastas apropriadas para treinamento, validação e teste. As imagens foram rotuladas com as classes `telhado` e `pessoa`.

Configuramos o arquivo `telhado.yaml`, que contém as definições das classes e os caminhos para os conjuntos de dados.

In [ ]:
! cp /content/drive/MyDrive/PROJETOS/DRONE/2024/telhado.yaml yolov5/data/

In [ ]:
!ls yolov5/data
#verificando conteudo

Argoverse.yaml	  GlobalWheat2020.yaml	ImageNet10.yaml  scripts	VOC.yaml
coco128-seg.yaml  hyps			ImageNet.yaml	 SKU-110K.yaml	xView.yaml
coco128.yaml	  ImageNet1000.yaml	images		 telhado.yaml
coco.yaml	  ImageNet100.yaml	Objects365.yaml  VisDrone.yaml


### Etapa 3: Treinamento do Modelo e Avaliação do Modelo
O modelo YOLOv5 foi treinado utilizando os dados preparados. Especificamos os parâmetros como número de épocas, tamanho da imagem e modelo base (yolov5s.pt).

Após o treinamento, avaliamos a performance do modelo utilizando métricas como precisão, recall e mAP. Os resultados foram visualizados para garantir a qualidade da detecção.


In [ ]:
! python yolov5/train.py --data telhado.yaml --weights yolov5s.pt --img 640 --epochs 40

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-10-24 18:03:49.373615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 18:03:49.398580: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 18:03:49.406529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

### Etapa 4: Aplicação do Modelo

Finalmente, aplicamos o modelo treinado para detectar objetos em novas imagens de teste, visualizando os resultados para confirmar a eficácia do modelo.

In [ ]:
import os
import subprocess

def get_latest_train_run_folder():
    subfolders = [f.path for f in os.scandir('yolov5/runs/train') if f.is_dir()]
    latest_folder = max(subfolders, key=os.path.getctime, default=None)
    return latest_folder

latest_run = get_latest_train_run_folder()
result = subprocess.run(f'python yolov5/detect.py --weights {latest_run}/weights/best.pt --img 640 --source drive/MyDrive/PROJETOS/DRONE/2024/test/ --data yolov5/data/telhado.yaml', shell=True, capture_output=True, text=True)
if latest_run:
    # COMANDO
    result = subprocess.run(f'python yolov5/detect.py --weights {latest_run}/weights/best.pt --img 640 --source drive/MyDrive/PROJETOS/DRONE/2024/test/ --data yolov5/data/telhado.yaml', shell=True, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
else:
    print("Não foi possível encontrar a pasta de treinamento mais recente.")


detect: weights=['yolov5/runs/train/exp/weights/best.pt'], source=drive/MyDrive/PROJETOS/DRONE/2024/test/, data=yolov5/data/telhado.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/MyDrive/PROJETOS/DRONE/2024/test/img22.jpeg: 640x480 3 telhados, 1 pessoa, 298.7ms
Speed: 2.3ms pre-process, 298.7ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5/runs/detect/exp2



In [ ]:
#listando diretorios
import os
print(os.getcwd())
print(os.listdir('yolov5/runs/train'))


/content
['exp']


### Etapa 5: Salvamento e Distribuição do Modelo
O arquivo `best.pt`, contendo os pesos do modelo treinado, foi salvo e pode ser utilizado para futuras detecções ou re-treinamento.

In [ ]:

#yolov5/runs/train/exp
#caminho anterior: yolov5/runs/train/exp4/
from google.colab import files
files.download('yolov5/runs/train/exp/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>